# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 5 2022 10:53AM,243542,15,ALI395494,"Alliance Pharma, Inc.",Released
1,Aug 5 2022 10:53AM,243542,15,30009789,"Alliance Pharma, Inc.",Released
2,Aug 5 2022 10:53AM,243542,15,30009788,"Alliance Pharma, Inc.",Released
3,Aug 5 2022 10:53AM,243542,15,30009790,"Alliance Pharma, Inc.",Released
4,Aug 5 2022 10:53AM,243542,15,30009791,"Alliance Pharma, Inc.",Released
5,Aug 5 2022 10:53AM,243542,15,30009793,"Alliance Pharma, Inc.",Released
6,Aug 5 2022 10:53AM,243542,15,30009792,"Alliance Pharma, Inc.",Released
7,Aug 5 2022 10:53AM,243542,15,30009794,"Alliance Pharma, Inc.",Released
8,Aug 5 2022 10:53AM,243542,15,30009795,"Alliance Pharma, Inc.",Released
9,Aug 5 2022 10:53AM,243542,15,30009796,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,243540,Released,32
34,243541,Released,20
35,243542,Released,14
36,243545,Released,1
37,243546,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243540,NaN,NaN,32.0
243541,NaN,NaN,20.0
243542,NaN,NaN,14.0
243545,NaN,NaN,1.0
243546,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243451,0.0,1.0,0.0
243455,0.0,0.0,1.0
243461,1.0,18.0,0.0
243464,0.0,7.0,6.0
243478,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243451,0,1,0
243455,0,0,1
243461,1,18,0
243464,0,7,6
243478,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243451,0,1,0
1,243455,0,0,1
2,243461,1,18,0
3,243464,0,7,6
4,243478,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243451,,1,
1,243455,,,1
2,243461,1,18,
3,243464,,7,6
4,243478,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 5 2022 10:53AM,243542,15,"Alliance Pharma, Inc."
14,Aug 5 2022 10:51AM,243541,20,"ACI Healthcare USA, Inc."
34,Aug 5 2022 10:48AM,243546,19,"Graystone, LLC"
35,Aug 5 2022 10:48AM,243540,20,"ACI Healthcare USA, Inc."
67,Aug 5 2022 10:46AM,243545,21,"NBTY Global, Inc."
68,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd"
70,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.
71,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc"
75,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.
76,Aug 5 2022 9:32AM,243535,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 5 2022 10:53AM,243542,15,"Alliance Pharma, Inc.",,,14
1,Aug 5 2022 10:51AM,243541,20,"ACI Healthcare USA, Inc.",,,20
2,Aug 5 2022 10:48AM,243546,19,"Graystone, LLC",,,1
3,Aug 5 2022 10:48AM,243540,20,"ACI Healthcare USA, Inc.",,,32
4,Aug 5 2022 10:46AM,243545,21,"NBTY Global, Inc.",,,1
5,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd",,1,1
6,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.,,,1
7,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc",,1,3
8,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.,,,1
9,Aug 5 2022 9:32AM,243535,12,Hush Hush,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 10:53AM,243542,15,"Alliance Pharma, Inc.",14,,
1,Aug 5 2022 10:51AM,243541,20,"ACI Healthcare USA, Inc.",20,,
2,Aug 5 2022 10:48AM,243546,19,"Graystone, LLC",1,,
3,Aug 5 2022 10:48AM,243540,20,"ACI Healthcare USA, Inc.",32,,
4,Aug 5 2022 10:46AM,243545,21,"NBTY Global, Inc.",1,,
5,Aug 5 2022 10:14AM,243539,19,"AdvaGen Pharma, Ltd",1,1,
6,Aug 5 2022 10:01AM,243538,16,Sartorius Stedim Filters Inc.,1,,
7,Aug 5 2022 9:58AM,243537,12,"Uniderm USA, Inc",3,1,
8,Aug 5 2022 9:48AM,243536,16,Sartorius Stedim Filters Inc.,1,,
9,Aug 5 2022 9:32AM,243535,12,Hush Hush,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 10:53AM,243542,15,"Alliance Pharma, Inc.",14,,
1,Aug 5 2022 10:51AM,243541,20,"ACI Healthcare USA, Inc.",20,,
2,Aug 5 2022 10:48AM,243546,19,"Graystone, LLC",1,,
3,Aug 5 2022 10:48AM,243540,20,"ACI Healthcare USA, Inc.",32,,
4,Aug 5 2022 10:46AM,243545,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 10:53AM,243542,15,"Alliance Pharma, Inc.",14.0,NaN,NaN
1,Aug 5 2022 10:51AM,243541,20,"ACI Healthcare USA, Inc.",20.0,NaN,NaN
2,Aug 5 2022 10:48AM,243546,19,"Graystone, LLC",1.0,NaN,NaN
3,Aug 5 2022 10:48AM,243540,20,"ACI Healthcare USA, Inc.",32.0,NaN,NaN
4,Aug 5 2022 10:46AM,243545,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 5 2022 10:53AM,243542,15,"Alliance Pharma, Inc.",14.0,0.0,0.0
1,Aug 5 2022 10:51AM,243541,20,"ACI Healthcare USA, Inc.",20.0,0.0,0.0
2,Aug 5 2022 10:48AM,243546,19,"Graystone, LLC",1.0,0.0,0.0
3,Aug 5 2022 10:48AM,243540,20,"ACI Healthcare USA, Inc.",32.0,0.0,0.0
4,Aug 5 2022 10:46AM,243545,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1948027,45.0,11.0,0.0
12,730569,3.0,3.0,0.0
15,487051,15.0,0.0,0.0
16,1461164,6.0,0.0,0.0
19,730536,2.0,2.0,0.0
20,1704561,55.0,26.0,1.0
21,487028,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1948027,45.0,11.0,0.0
1,12,730569,3.0,3.0,0.0
2,15,487051,15.0,0.0,0.0
3,16,1461164,6.0,0.0,0.0
4,19,730536,2.0,2.0,0.0
5,20,1704561,55.0,26.0,1.0
6,21,487028,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,45.0,11.0,0.0
1,12,3.0,3.0,0.0
2,15,15.0,0.0,0.0
3,16,6.0,0.0,0.0
4,19,2.0,2.0,0.0
5,20,55.0,26.0,1.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,45.0
1,12,Released,3.0
2,15,Released,15.0
3,16,Released,6.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Executing,11.0,3.0,0.0,0.0,2.0,26.0,1.0
Released,45.0,3.0,15.0,6.0,2.0,55.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,11.0,3.0,0.0,0.0,2.0,26.0,1.0
2,Released,45.0,3.0,15.0,6.0,2.0,55.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,11.0,3.0,0.0,0.0,2.0,26.0,1.0
2,Released,45.0,3.0,15.0,6.0,2.0,55.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()